In [ ]:
import numpy as np
import sys
sys.path.append('/home/mariusvaardal/AAMAS_project/AAMAS_project')
import math
import utils.utils as utils
from pettingzoo.mpe import simple_tag_v3

from agent_types.CoordinatingAgent import CoordinatingAgent
from agent_types.GreedyAgent import GreedyAgent
from agent_types.RandomAgent import RandomAgent
from agent_types.AvoidingAgent import AvoidingAgent
from agent_types.AvoidingNearestAdversaryAgent import AvoidingNearestAdversaryAgent
from agent_types.ImmobileAgent import ImmobileAgent
from agent_types.RLAgent_2_adv import RLAgent


NUM_EPISODES = 50
NUM_GOOD = 1
NUM_ADVERSARIES = 2
NUM_LANDMARKS = 0
MAX_CYCLES = 200

THRESHOLD = 1e-5
DISTANCE_THRESHOLD = 0.5
TARGET_VECT_THRESHOLD = 5

RENDER_MODE = 'human'

env = simple_tag_v3.parallel_env(num_good=NUM_GOOD, num_adversaries=NUM_ADVERSARIES, num_obstacles=NUM_LANDMARKS, max_cycles=MAX_CYCLES, continuous_actions=False, render_mode=RENDER_MODE)
env.reset()
# coord_agents = [CoordinatingAgent(name, NUM_ADVERSARIES, NUM_LANDMARKS) for name in env.agents if name != "agent_0"]
# coord_agents = [GreedyAgent(name, NUM_ADVERSARIES, NUM_LANDMARKS) for name in env.agents if name != "agent_0"]
coord_agents = [RLAgent(name, NUM_ADVERSARIES, NUM_LANDMARKS) for name in env.agents if name != "agent_0"]
# coord_agents = [ImmobileAgent(name, NUM_ADVERSARIES, NUM_LANDMARKS) for name in env.agents if name != "agent_0"]
coord_agents.append(AvoidingAgent("agent_0", NUM_ADVERSARIES, NUM_LANDMARKS))
# coord_agents.append(AvoidingNearestAdversaryAgent("agent_0", NUM_ADVERSARIES, NUM_LANDMARKS))
# coord_agents.append(RandomAgent("agent_0", NUM_ADVERSARIES, NUM_LANDMARKS))
# coord_agents.append(ImmobileAgent("agent_0", NUM_ADVERSARIES, NUM_LANDMARKS))

# this implementation assumes one prey only

episode_rewards = []
for episode in range(1, NUM_EPISODES + 1):
    observations, infos = env.reset()
    for agent in coord_agents:
        agent.observations = observations[agent.name]
        agent.update_observed_agent_positions()
        
    episode_reward = 0
    while env.agents:
        # print(f"Agent pos : {coord_agents[0].observed_agent_positions['agent_0']}")
        actions = {}
        for agent in coord_agents:
            action = agent.get_action()
            actions[agent.name] = action if action != None else 0
        observations, rewards, terminations, truncations, infos = env.step(actions)
        for agent in coord_agents:
            # agent.see(observations[agent.name])
            agent.observations = observations[agent.name]
            agent.update_observed_agent_positions()
        episode_reward += utils.get_timestep_reward(rewards)
        # print(f"Agent_0 pos: {coord_agents[0].observed_agent_positions['agent_0']}")
    episode_rewards.append(episode_reward)
    # print(f"Episode: {episode}. Reward: {sum(episode_rewards)}")
print(f"Avg : {sum(episode_rewards) / len(episode_rewards)}. For {len(episode_rewards)} number of episodes")

env.close()










In [2]:
env.close()